In [26]:
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords

In [23]:
df=pd.read_excel("G:/My Drive/Maestría/MCD-TPs-Grupo/Test Mining/full_notas.xlsx")

In [13]:
df.head(5)

,titulo,descripcion,link,fecha,diario,contenido
0,Qué es el “criptoinvierno” y por qué el bitcoi...,Desde el inicio de este año muchos analistas v...,https://www.lanacion.com.ar/economia/que-es-el...,44690.927083,LaNacion,"Cuando las cosas van bien, en la jerga financi..."
1,Tragedia en Palermo. Cómo diferenciar cuando l...,Identificarlos a tiempo puede ayudar a salvar ...,https://www.lanacion.com.ar/salud/mente/traged...,44690.926389,LaNacion,El sábado pasado un taxista se descompensó mie...
2,Horacio Rodríguez Larreta reclamó “ampliar” Ju...,"Con Mauricio Macri sentado a pocos metros, el ...",https://www.lanacion.com.ar/politica/horacio-r...,44690.923611,LaNacion,"El jefe de gobierno porteño, Horacio Rodríguez..."
3,Las diez obras de arte más caras vendidas en s...,Shot Sage Blue Marilyn de Andy Warhol quedó se...,https://www.lanacion.com.ar/cultura/las-diez-o...,44690.922222,LaNacion,"Shot Sage Blue Marilyn de Andy Warhol, vendida..."
4,"Mauricio Macri, Mario Vargas Llosa y Julio Mar...","Del encuentro, en el complejo Goldcenter de Pa...",https://www.lanacion.com.ar/politica/mauricio-...,44690.921528,LaNacion,La Fundación Libertad celebra esta noche su ce...


In [14]:
df.tail()

,titulo,descripcion,link,fecha,diario,contenido
3494,España dejará de exigir certificado de vacunac...,NaN,https://www.pagina12.com.ar/422900-espana-deja...,2022-05-19 18:37:57,Pagina12,El gobierno de España anunció esste jueves que...
3495,La OMS autorizó la vacuna china de CanSino,NaN,https://www.pagina12.com.ar/422896-la-oms-auto...,2022-05-19 18:13:36,Pagina12,La Organización Mundial de la Salud (OMS) otor...
3496,Varios países registran casos de viruela del m...,NaN,https://www.pagina12.com.ar/422894-varios-pais...,2022-05-19 18:11:57,Pagina12,Estados Unidos y Europa detectaron decenas de ...
3497,Casi unos 200 kilos de cocaína fueron encontra...,El hallazgo de la droga tiene un antecedente: ...,https://www.diariocronica.com.ar/noticias/2022...,2022-05-19 19:32:25,Cronica,Casi unos 200 kilos de cocaína fueron encontra...
3498,Una mujer asesinó a su hija de 6 años y se ent...,El filicidio ocurrió el miércoles en la locali...,https://www.diariocronica.com.ar/noticias/2022...,2022-05-19 19:29:11,Cronica,Una mujer asesinó a su hija de 6 años y se ent...


In [15]:
print("Cantidad de targets: ")
df.diario.value_counts()

Cantidad de targets: 


Cronica     1286
Clarin       716
Telam        701
LaNacion     511
Pagina12     285
Name: diario, dtype: int64

In [22]:
import re
import string
#pasamos a minuscula y quitamos signos de puntuacion, pregunta, exclamacion, enter, tabs, etc
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text

fx_clean = lambda x: clean_text(x)

In [24]:
data_clean = pd.DataFrame(df.contenido.apply(fx_clean))
data_clean

,contenido
0,cuando las cosas van bien en la jerga financi...
1,el sábado pasado un taxista se descompensó mie...
2,el jefe de gobierno porteño horacio rodríguez...
3,shot sage blue marilyn de andy warhol vendida...
4,la fundación libertad celebra esta noche su ce...
...,...
3487,el gobierno de españa anunció esste jueves que...
3488,la organización mundial de la salud oms otor...
3489,estados unidos y europa detectaron decenas de ...
3490,casi unos kilos de cocaína fueron encontrados...


In [36]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VEIGC901\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VEIGC901\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
stop_words = set(stopwords.words('spanish'))

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_clean.contenido)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index

data_dtm.to_pickle("dtm.pkl")
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

c:\Users\VEIGC901\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [39]:
data_dtm

,aa,aaa,aaaj,aabe,aacute,aago,aaici,aamf,aaoc,aaoncoclinica,...,úrsula,úsuga,útero,útil,útiles,útima,útimo,útimos,špor,𝔇𝔞𝔫𝔦𝔢𝔩𝔞
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3487,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3488,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3489,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3490,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
